# **Lab 3: Categorical Data**

Experiment on multivariate linear regression with feature engineering or with any algorithms of your choice.

## Load and Prepare Datasets


In [1]:
#import packages
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

#Train file URL Link
csv_url_train = "https://raw.githubusercontent.com/shalimarchalhoub/Regression-Assignment/main/cancer_us_county-training.csv"

#Test file URL Link
csv_url_test = "https://raw.githubusercontent.com/shalimarchalhoub/Regression-Assignment/main/cancer_us_county-testing.csv"

#Load Train data into DataFrame df_train
df_train = pd.read_csv(csv_url_train)

#Load Test Data into Datframe df_test
df_test = pd.read_csv(csv_url_test)


## Data Preparation

Pre Processing Train Dataset

In [2]:
#Selecting Numeric Columns
numeric_cols = df_train.select_dtypes(include='number').columns
df_train_x = df_train.loc[:, numeric_cols]

#Dropping the target Variable from x_train
df_train_x.drop('TARGET_deathRate',axis=1, inplace=True)


x_train = df_train_x #predictor dataset

#Select the target variable for y_train
y_train = df_train['TARGET_deathRate'] #target variable 


Pre processing test dataset

In [3]:
#Selecting Numeric Columns
numeric_cols_t = df_test.select_dtypes(include='number').columns
df_test_x = df_test.loc[:, numeric_cols_t]
#Dropping the target Variable from x_test
df_test_x.drop('TARGET_deathRate',axis=1, inplace=True)

x_test = df_test_x #predictor dataset

#select the target variable for y_test
y_test = df_test['TARGET_deathRate'].values #target variable 


Check the first 5 rows of x_train as well as the summary of it

In [4]:
#First 5 rows of x_train
x_train.head()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,...,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Id
0,88.000000,40,561.400000,29090,13352,26.8,2771.120431,39.8,39.2,40.5,...,32.0,47.5,32.9,99.693045,0.044920,0.000000,0.000000,55.499459,6.838710,0
1,73.000000,35,345.600000,29782,21903,38.8,0.000000,32.3,30.8,35.2,...,18.8,45.3,34.1,94.791383,1.649850,0.063631,2.854286,52.818296,4.799131,1
2,292.000000,124,468.400000,41955,48985,15.5,0.000000,42.2,40.9,43.8,...,44.9,34.5,16.0,95.102348,1.741749,0.376429,0.445611,50.560800,3.996826,2
3,1962.667684,7,453.549422,55378,3007,11.1,0.000000,41.6,38.3,46.3,...,49.6,30.1,15.2,85.833870,0.933677,0.160979,7.244044,52.565181,3.291536,3
4,43.000000,20,349.700000,26309,8551,35.3,0.000000,43.9,41.2,47.8,...,30.4,45.1,24.5,24.535525,73.223736,0.394100,1.396239,33.641208,3.166561,4


In [5]:
#Summary of x_train
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              2438 non-null   float64
 1   avgDeathsPerYear         2438 non-null   int64  
 2   incidenceRate            2438 non-null   float64
 3   medIncome                2438 non-null   int64  
 4   popEst2015               2438 non-null   int64  
 5   povertyPercent           2438 non-null   float64
 6   studyPerCap              2438 non-null   float64
 7   MedianAge                2438 non-null   float64
 8   MedianAgeMale            2438 non-null   float64
 9   MedianAgeFemale          2438 non-null   float64
 10  AvgHouseholdSize         2438 non-null   float64
 11  PercentMarried           2438 non-null   float64
 12  PctNoHS18_24             2438 non-null   float64
 13  PctHS18_24               2438 non-null   float64
 14  PctSomeCol18_24         

Check the first 5 rows of x_test as well as the summary of it

In [6]:
#First 5 rows of x_test
x_test.head()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,...,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Id
0,449.0,154,479.8,51880,104926,18.7,57.183158,30.2,28.5,31.8,...,51.0,24.9,13.1,81.260411,4.154831,10.045737,0.876222,41.071243,4.367123,2553
1,340.0,140,438.5,55472,55423,12.4,0.000000,46.9,44.9,48.2,...,37.6,36.3,16.0,93.660078,0.818115,0.626281,3.116360,57.529142,6.844366,904
2,54.0,18,410.8,49380,10103,11.7,0.000000,49.4,49.4,49.4,...,32.6,40.3,19.4,98.292181,0.041152,0.164609,0.051440,55.928482,1.604585,2192
3,94.0,46,403.8,45979,16708,13.5,598.515681,43.9,42.1,45.0,...,45.1,33.1,13.3,96.090377,1.555569,0.715680,0.378541,48.409405,8.255410,1326
4,2718.0,1065,432.1,51527,726106,20.7,60.597213,33.5,32.4,34.6,...,41.7,37.7,25.8,57.002148,7.093743,14.785464,11.692122,51.852122,6.148433,2394


In [7]:
#Summary of x_test
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              609 non-null    float64
 1   avgDeathsPerYear         609 non-null    int64  
 2   incidenceRate            609 non-null    float64
 3   medIncome                609 non-null    int64  
 4   popEst2015               609 non-null    int64  
 5   povertyPercent           609 non-null    float64
 6   studyPerCap              609 non-null    float64
 7   MedianAge                609 non-null    float64
 8   MedianAgeMale            609 non-null    float64
 9   MedianAgeFemale          609 non-null    float64
 10  AvgHouseholdSize         609 non-null    float64
 11  PercentMarried           609 non-null    float64
 12  PctNoHS18_24             609 non-null    float64
 13  PctHS18_24               609 non-null    float64
 14  PctSomeCol18_24          1

Fixing Missing values and Dropping irrelevant columns

In [8]:
# Dropping the PctSomeCol18_24 column since more than 3/4 of its values are missing in the train dataset
x_train.drop(['PctSomeCol18_24'], axis=1, inplace= True)
x_test.drop(['PctSomeCol18_24'], axis=1, inplace= True)

#Removing the Id columns
x_test.drop("Id",axis=1, inplace=True)
x_train.drop("Id",axis=1, inplace=True)

In [9]:
#Replacing Missing values in PctEmployed16_Over and PctPrivateCoverageAlone with the mean in both train and test
PctEmployed16_Over_mean = x_train['PctEmployed16_Over'].mean()
x_train['PctEmployed16_Over'] = x_train['PctEmployed16_Over'].fillna(PctEmployed16_Over_mean)

PctPrivateCoverageAlone_mean = x_train['PctPrivateCoverageAlone'].mean()
x_train['PctPrivateCoverageAlone'] = x_train['PctPrivateCoverageAlone'].fillna(PctPrivateCoverageAlone_mean)

#For the test dataset
employed_mean_test = x_test['PctEmployed16_Over'].mean()
x_test['PctEmployed16_Over'] = x_test['PctEmployed16_Over'].fillna(employed_mean_test)

Coverage_mean = x_test['PctPrivateCoverageAlone'].mean()
x_test['PctPrivateCoverageAlone'] = x_test['PctPrivateCoverageAlone'].fillna(Coverage_mean)

In [10]:
# Making sure missing values are gone from train dataset
x_train.isna().sum()

avgAnnCount                0
avgDeathsPerYear           0
incidenceRate              0
medIncome                  0
popEst2015                 0
povertyPercent             0
studyPerCap                0
MedianAge                  0
MedianAgeMale              0
MedianAgeFemale            0
AvgHouseholdSize           0
PercentMarried             0
PctNoHS18_24               0
PctHS18_24                 0
PctBachDeg18_24            0
PctHS25_Over               0
PctBachDeg25_Over          0
PctEmployed16_Over         0
PctUnemployed16_Over       0
PctPrivateCoverage         0
PctPrivateCoverageAlone    0
PctEmpPrivCoverage         0
PctPublicCoverage          0
PctPublicCoverageAlone     0
PctWhite                   0
PctBlack                   0
PctAsian                   0
PctOtherRace               0
PctMarriedHouseholds       0
BirthRate                  0
dtype: int64

In [11]:
# Making sure missing values are gone from test dataset
x_test.isna().sum()

avgAnnCount                0
avgDeathsPerYear           0
incidenceRate              0
medIncome                  0
popEst2015                 0
povertyPercent             0
studyPerCap                0
MedianAge                  0
MedianAgeMale              0
MedianAgeFemale            0
AvgHouseholdSize           0
PercentMarried             0
PctNoHS18_24               0
PctHS18_24                 0
PctBachDeg18_24            0
PctHS25_Over               0
PctBachDeg25_Over          0
PctEmployed16_Over         0
PctUnemployed16_Over       0
PctPrivateCoverage         0
PctPrivateCoverageAlone    0
PctEmpPrivCoverage         0
PctPublicCoverage          0
PctPublicCoverageAlone     0
PctWhite                   0
PctBlack                   0
PctAsian                   0
PctOtherRace               0
PctMarriedHouseholds       0
BirthRate                  0
dtype: int64

In [12]:
#Checking the corrolation of my data
corr_matrix = x_train.corr()
corr_df = corr_matrix.reset_index().melt(id_vars='index')

heatmap = alt.Chart(corr_df).mark_rect().encode(
    x=alt.X('variable:N', title=''),
    y=alt.Y('index:N', title=''),
    color=alt.Color('value:Q', scale=alt.Scale(scheme='redyellowblue'))
)
heatmap

alt.Chart(...)

In [13]:
#Creating a new variable which consists of the avgDeathsPerYear devided by population and multiplied by 100 000
#This is done in order to try and replicate the Target Variable in theory
x_train['new'] = x_train['avgDeathsPerYear']*100000/x_train['popEst2015']

#Same for the test set
x_test['new'] = x_test['avgDeathsPerYear']*100000/x_test['popEst2015']

Splitting the train dataset into train and dev

In [14]:
X_train, X_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

## Training a Lasso Model

Instantiating a Lasso model into a variable called `model` and fitting in on the data

### Train Data

In [15]:
#Initiating a Ridge model
model = Ridge()

#Fitting the data to the model
model.fit(X_train, y_train)

#Making Predictions
y_train_preds = model.predict(X_train)

#Checking the Mean Squared Error
print("RMSE is" , mse(y_train, y_train_preds, squared = False))
print("MSE is" , mse(y_train, y_train_preds))

RMSE is 12.4333887477547
MSE is 154.58915575279318


In [16]:
# Graph of perfect values versus predictions
perfect_train = alt.Chart(pd.DataFrame({'target': y_train, 'preds': y_train})).mark_line(color='green').encode(
    x='target',
    y='preds'
)

pred_chart_train = alt.Chart(pd.DataFrame({'target': y_train, 'preds': y_train_preds})).mark_line().encode(
    x='target',
    y='preds'
  )

pred_chart_train + perfect_train

alt.LayerChart(...)

### Validation Model

In [17]:
#Testing the validation model
y_dev_pred = model.predict(X_dev)

#Checking the Mean Squared Error
print("RMSE is" , mse(y_dev, y_dev_pred, squared = False))
print("MSE is" , mse(y_dev, y_dev_pred))

RMSE is 12.547515857956578
MSE is 157.44015420567177


In [18]:
# Graph of perfect values versus predictions
perfect_dev = alt.Chart(pd.DataFrame({'target': y_dev, 'preds': y_dev})).mark_line(color='green').encode(
    x='target',
    y='preds'
)

pred_chart_dev = alt.Chart(pd.DataFrame({'target': y_dev, 'preds': y_dev_pred})).mark_line().encode(
    x='target',
    y='preds'
  )

pred_chart_dev + perfect_dev

alt.LayerChart(...)

### Test Set

In [19]:
#Testing the validation model
y_test_pred = model.predict(x_test)

#Checking the Mean Squared Error
print("RMSE is" , mse(y_test, y_test_pred, squared = False))
print("MSE is" , mse(y_test, y_test_pred))

RMSE is 12.886254201274962
MSE is 166.0555473398766


In [20]:
# Graph of perfect values versus predictions
perfect_test = alt.Chart(pd.DataFrame({'target': y_test, 'preds': y_test})).mark_line(color='green').encode(
    x='target',
    y='preds'
)

pred_chart_test = alt.Chart(pd.DataFrame({'target': y_test, 'preds': y_test_pred})).mark_line().encode(
    x='target',
    y='preds'
  )

pred_chart_test + perfect_test

alt.LayerChart(...)